# helper on anomarly detection experiments

In [1]:
import os, sys
import numpy as np
import torch
import json

sys.path.append("..")
from singleVis.SingleVisualizationModel import SingleVisualizationModel
from singleVis.data import NormalDataProvider
from singleVis.eval.evaluator import Evaluator
from singleVis.projector import Projector

In [2]:
dataset = "symmetric"
CONTENT_PATH ="/home/xianglin/projects/DVI_data/noisy/{}/10/".format(dataset)

In [3]:
sys.path.append(CONTENT_PATH)
from config import config

In [7]:
GPU_ID = 0
SETTING = config["SETTING"]
CLASSES = config["CLASSES"]
DATASET = config["DATASET"]
PREPROCESS = config["VISUALIZATION"]["PREPROCESS"]
EPOCH_START = config["EPOCH_START"]
EPOCH_END = config["EPOCH_END"]
EPOCH_PERIOD = config["EPOCH_PERIOD"]

# Training parameter (subject model)
TRAINING_PARAMETER = config["TRAINING"]
NET = TRAINING_PARAMETER["NET"]
LEN = TRAINING_PARAMETER["train_num"]

# Training parameter (visualization model)
VISUALIZATION_PARAMETER = config["VISUALIZATION"]
LAMBDA = VISUALIZATION_PARAMETER["LAMBDA"]
S_LAMBDA = VISUALIZATION_PARAMETER["S_LAMBDA"]
B_N_EPOCHS = VISUALIZATION_PARAMETER["BOUNDARY"]["B_N_EPOCHS"]
L_BOUND = VISUALIZATION_PARAMETER["BOUNDARY"]["L_BOUND"]
INIT_NUM = VISUALIZATION_PARAMETER["INIT_NUM"]
ALPHA = VISUALIZATION_PARAMETER["ALPHA"]
BETA = VISUALIZATION_PARAMETER["BETA"]
MAX_HAUSDORFF = VISUALIZATION_PARAMETER["MAX_HAUSDORFF"]
HIDDEN_LAYER = VISUALIZATION_PARAMETER["HIDDEN_LAYER"]
S_N_EPOCHS = VISUALIZATION_PARAMETER["S_N_EPOCHS"]
T_N_EPOCHS = VISUALIZATION_PARAMETER["T_N_EPOCHS"]
N_NEIGHBORS = VISUALIZATION_PARAMETER["N_NEIGHBORS"]
PATIENT = VISUALIZATION_PARAMETER["PATIENT"]
MAX_EPOCH = VISUALIZATION_PARAMETER["MAX_EPOCH"]
SEGMENTS = VISUALIZATION_PARAMETER["SEGMENTS"]
RESUME_SEG = VISUALIZATION_PARAMETER["RESUME_SEG"]
# define hyperparameters
DEVICE = torch.device("cuda:{}".format(GPU_ID) if torch.cuda.is_available() else "cpu")

content_path = CONTENT_PATH
sys.path.append(content_path)

import Model.model as subject_model
# net = resnet18()
net = eval("subject_model.{}()".format(NET))
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

In [8]:
data_provider = NormalDataProvider(CONTENT_PATH, net, EPOCH_START, EPOCH_END, EPOCH_PERIOD, split=-1, device=DEVICE, classes=CLASSES,verbose=1)
# if PREPROCESS:
#     data_provider.initialize(LEN//10, l_bound=L_BOUND)

model = SingleVisualizationModel(input_dims=512, output_dims=2, units=256, hidden_layer=HIDDEN_LAYER)
projector = Projector(vis_model=model, content_path=CONTENT_PATH, segments=SEGMENTS, device=DEVICE)

Finish initialization...


In [9]:
from singleVis.visualizer import visualizer
vis = visualizer(data_provider, projector, 200)

In [10]:
path = "/home/xianglin/projects/DVI_data/noisy/{}/10/clean_label.json".format(dataset)
with open(path, "r") as f:
    clean_label = json.load(f)
path = "/home/xianglin/projects/DVI_data/noisy/{}/10/noisy_label.json".format(dataset)
with open(path, "r") as f:
    noisy_label = json.load(f)
clean_label = np.array(clean_label)
noisy_label = np.array(noisy_label)

In [ ]:
train_labels = data_provider.train_labels(40)

In [11]:
samples = np.zeros((200, 50000, 512))
for i in range(1, 201, 1):
    samples[i-1] = data_provider.train_representation(i)

In [12]:
embeddings_2d = np.zeros((200, 50000, 2))
for e in range(1, 201, 1):
    embeddings_2d[e-1] = projector.batch_project(e, samples[e-1])
embeddings_2d = np.transpose(embeddings_2d, [1,0,2])

Successfully load the visualization model for range (1,32)...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization model...
Same range as current visualization mo

In [13]:
path = "/home/xianglin/projects/DVI_data/noisy/{}/10/embedding.npy".format(dataset)
np.save(path, embeddings_2d)

In [16]:
embeddings = embeddings_2d[:,80:,:]

In [17]:
v = embeddings[:,1:,:]-embeddings[:,:-1,:]
a = v[:,1:,:]-v[:,:-1,:]
v.shape, a.shape

((50000, 119, 2), (50000, 118, 2))

In [18]:
from sklearn import metrics
from sklearn.cluster import Birch
brc = Birch(n_clusters=30)
brc.fit(a.reshape(50000, -1))

Birch(n_clusters=30)

In [19]:
c = 30
gt = np.argwhere(noisy_label!=clean_label).squeeze()
rates = np.zeros(c)
wrong = np.zeros(c)
sel = np.zeros(c)
for suspect in range(c):
    selected_idxs = np.argwhere(brc.labels_==suspect).squeeze()
    wrong[suspect] = len(np.intersect1d(selected_idxs, gt))
    sel[suspect] = len(selected_idxs)
    rates[suspect] = len(np.intersect1d(selected_idxs, gt))/len(selected_idxs)

In [20]:
ranking = np.argsort(rates)
for r in range(c-1,-1,-1):
    print(round(rates[ranking[r]], 3),"\t", int(wrong[ranking[r]]),"/", int(sel[ranking[r]]))

0.406 	 183 / 451
0.327 	 205 / 627
0.297 	 251 / 846
0.241 	 127 / 527
0.205 	 76 / 370
0.192 	 74 / 386
0.184 	 190 / 1031
0.168 	 51 / 303
0.16 	 56 / 349
0.155 	 50 / 323
0.149 	 48 / 323
0.12 	 184 / 1527
0.111 	 85 / 769
0.105 	 67 / 637
0.101 	 131 / 1296
0.099 	 60 / 608
0.092 	 54 / 587
0.086 	 88 / 1022
0.085 	 54 / 633
0.083 	 1576 / 18896
0.082 	 65 / 794
0.082 	 28 / 343
0.081 	 47 / 579
0.08 	 34 / 423
0.077 	 184 / 2387
0.074 	 77 / 1037
0.068 	 736 / 10828
0.062 	 25 / 400
0.05 	 50 / 995
0.05 	 35 / 703


In [21]:
noise_idxs = np.argwhere(clean_label!=noisy_label).squeeze()
selected = np.random.choice(noise_idxs,size=300,replace=False)
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(a.reshape(50000, -1))
_, idxs = nbrs.kneighbors(a.reshape(50000, -1)[selected])
selected_idxs = idxs[:,1:].reshape(-1)
selected_idxs = np.unique(selected_idxs)

In [22]:
len(np.intersect1d(selected_idxs, noise_idxs)), len(selected_idxs)

(50, 297)

In [24]:
tr = samples[-1]

In [25]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='ball_tree').fit(tr)
_, idxs = nbrs.kneighbors(tr[selected])
selected_idxs = idxs[:,1:].reshape(-1)
selected_idxs = np.unique(selected_idxs)

In [26]:
len(np.intersect1d(selected_idxs, noise_idxs)), len(selected_idxs)

(93, 297)

In [27]:
selected = np.random.choice(50000,size=300,replace=False)

In [29]:
len(np.intersect1d(selected, noise_idxs)), len(selected_idxs)

(35, 297)

In [ ]:
path = "/home/xianglin/projects/DVI_data/noisy/{}/cifar10/embedding.npy".format(dataset)
embeddings_2d = np.load(path)

In [ ]:
path = "/home/xianglin/projects/DVI_data/resnet18_cifar10/embedding.npy"
embeddings_2d = np.load(path)

In [14]:
import matplotlib.pyplot as plt
import umap.umap_ as umap
from sklearn.manifold import TSNE 
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import Birch, KMeans

In [ ]:
def select_centroid(samples, n_select=3):
    kmeans = KMeans(n_clusters=n_select).fit(samples)

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(samples)
    indices = nbrs.kneighbors(kmeans.cluster_centers_,return_distance=False)
    return indices.squeeze()

In [ ]:
def test_abnormal(directions, repeat=2, show=True, dr="umap"):
    for _ in range(repeat):
        if dr=="umap":
            reducer = umap.UMAP(n_components=2)
            embedding = reducer.fit_transform(directions)
        elif dr=="tsne":
            embedding = TSNE(n_components=2, learning_rate='auto', init='pca').fit_transform(high_data)
        else:
            print("illgal dr algorithm!")
            return False

        brc = Birch(n_clusters=2)
        brc.fit(embedding)

        s = silhouette_score(embedding, brc.labels_, metric='euclidean')
        c = calinski_harabasz_score(embedding, brc.labels_)
        if s <= 0.1:
            continue
        else:
            break
    if s <= 0.1:
        print("No abnormal trajectories detected!")
        return False
    print("silhouette_score\t", s)
    print("calinski_harabasz_score\t", c)

    labels = brc.labels_
    centroid = brc.subcluster_centers_
    centroid_labels = brc.subcluster_labels_
    # clean 1, noise 0
    bin = np.bincount(centroid_labels)
    if bin[0] > bin[1]:
        centroid_labels = np.abs(centroid_labels-1)
        labels = np.abs(labels-1)
    
    # select 3 representative clean ones
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(embedding)
    indices = nbrs.kneighbors(centroid, return_distance=False)
    centroid = embedding[indices.squeeze()]

    # calculate noise score
    clean_centroids = centroid[centroid_labels==1]
    clean_ones = embedding[labels==1]
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(clean_centroids)
    dists, _ = nbrs.kneighbors(clean_ones)
    norm_term = dists.max()

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(clean_centroids)
    dists, _ = nbrs.kneighbors(embedding)
    scores = dists/norm_term
    
    if show:
        plt.scatter(
            embedding[:, 0],
            embedding[:, 1],
            s=1,
            c=labels,
            cmap="Pastel2")

        c_idxs = select_centroid(clean_centroids)
        plt.scatter(
            clean_centroids[c_idxs][:, 0],
            clean_centroids[c_idxs][:, 1],
            s=4,
            c="black",
            )
        plt.scatter(
            centroid[centroid_labels==0][:, 0],
            centroid[centroid_labels==0][:, 1],
            s=4,
            c="red",
            )
        plt.title('Trajectories Visualization', fontsize=24)
        plt.show()

    return labels, scores, centroid, centroid_labels, embedding

In [ ]:
# clean dataset
for cls_num in range(10):
    cls = np.argwhere(train_labels == cls_num).squeeze()
    # lbs = np.array(train_labels)[cls]
    high_data = directions[cls].reshape(len(cls), -1)
    test_abnormal(high_data)

In [ ]:
# clean dataset
for cls_num in range(10):
    cls = np.argwhere(train_labels == cls_num).squeeze()
    # lbs = np.array(train_labels)[cls]
    high_data = embeddings_2d[cls].reshape(len(cls), -1)
    _,_,_,_,embedding = test_abnormal(high_data)
    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=train_labels[cls],
        cmap="tab10")
    plt.show()

In [ ]:
# pairflip
for cls_num in range(10):
    cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
    # lbs = np.array(clean_label)[cls]
    high_data = embeddings_2d[cls].reshape(len(cls), -1)
    _,_,_,_,embedding = test_abnormal(high_data)
    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=clean_label[cls],
        cmap="tab10")
    plt.show()

In [ ]:
# symmetric
for cls_num in range(10):
    cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
    # lbs = np.array(clean_label)[cls]
    high_data = embeddings_2d[cls].reshape(len(cls), -1)
    _,_,_,_,embedding = test_abnormal(high_data, dr="umap")
    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=clean_label[cls],
        cmap="tab10")
    plt.show()
    

# noise score

In [ ]:
cls_num = 1
cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
high_data = directions[cls].reshape(len(cls), -1)

In [ ]:
cls_num = 1
cls = np.argwhere(train_labels == cls_num).squeeze()
high_data = directions[cls].reshape(len(cls), -1)

In [ ]:
reducer = umap.UMAP(n_components=2)
embedding = reducer.fit_transform(high_data)

brc = Birch(n_clusters=2)
brc.fit(embedding)

s = silhouette_score(embedding, brc.labels_, metric='euclidean')
c = calinski_harabasz_score(embedding, brc.labels_)
print("silhouette_score\t", s)
print("calinski_harabasz_score\t", c)

labels = brc.labels_
centroid = brc.subcluster_centers_
centroid_labels = brc.subcluster_labels_

In [ ]:
bin = np.bincount(centroid_labels)
if bin[0]>bin[1]:
    centroid_labels = np.abs(centroid_labels-1)
# 0 is noise
bin = np.bincount(labels)
if bin[0]>bin[1]:
    labels = np.abs(labels-1)

In [ ]:
clean_ones = embedding[labels==1]
noise_ones = embedding[labels==0]

In [ ]:
nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(centroid[centroid_labels==1])
dists, indices = nbrs.kneighbors(clean_ones)
norm = dists.max()
norm


In [ ]:
dists, indices = nbrs.kneighbors(noise_ones)
normed = dists/norm
normed.max(), normed.min(), normed.mean()

# ground truth

In [ ]:
for cls_num in range(10):
    cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
    high_data = embeddings_2d[cls].reshape(len(cls), -1)


    reducer = umap.UMAP(n_components=2)
    embedding = reducer.fit_transform(high_data)

    brc = Birch(n_clusters=2)
    brc.fit(embedding)

    labels = brc.labels_
    centroid = brc.subcluster_centers_
    centroid_labels = brc.subcluster_labels_
    # clean 1, noise 0
    bin = np.bincount(labels)
    if bin[0] > bin[1]:
        centroid_labels = np.abs(centroid_labels-1)
        labels = np.abs(labels-1)

    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=clean_label[cls],
        cmap="tab10")
    plt.scatter(
        brc.subcluster_centers_[:, 0],
        brc.subcluster_centers_[:, 1],
        s=5,
        c='black')
    plt.show()
    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=brc.labels_,
        cmap="Pastel2")

    nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centroid[centroid_labels==1])
    dists, indices = nbrs.kneighbors(centroid[centroid_labels==1])
    suspicious = (dists[:, -1]/ dists[:, 1])>1.8

    cleans = centroid[centroid_labels==1]
    noises = centroid[centroid_labels==0]
    plt.scatter(
        cleans[:, 0],
        cleans[:, 1],
        s=5,
        c='r')
    plt.scatter(
        noises[:, 0],
        noises[:, 1],
        s=5,
        c='black')
    plt.scatter(
        cleans[suspicious][:, 0],
        cleans[suspicious][:, 1],
        s=5,
        c='g')
    plt.show()

In [ ]:
from sklearn.mixture import GaussianMixture as GMM
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity

In [ ]:
s = 250
embedding_2d = embeddings_2d[:,s:]
l = (400-s)//2

In [ ]:
for cls_num in range(10):
    cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
    high_data = embedding_2d[cls].reshape(len(cls), -1)

    # umap reduction
    reducer = umap.UMAP(n_components=2)
    embedding = reducer.fit_transform(high_data)

    # PCA
    _, _, v = np.linalg.svd(high_data)
    scores = np.abs(np.inner(v[0], high_data))
    m = embedding.mean()
    scores = scores / scores.max()

    brc = Birch(n_clusters=2)
    brc.fit(embedding)
    s = silhouette_score(embedding, brc.labels_, metric='euclidean')
    c = calinski_harabasz_score(embedding, brc.labels_)
    print("silhouette_score\t", s)
    print("calinski_harabasz_score\t", c)
    labels = brc.labels_
    centroid = brc.subcluster_centers_
    centroid_labels = brc.subcluster_labels_

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(embedding)
    _, idxs = nbrs.kneighbors(centroid)
    centroid = embedding[idxs].squeeze(axis=1)

    # clean 0, noise 1
    if np.sum(labels==0)<np.sum(labels==1):
        labels = np.abs(1-labels)
        centroid_labels = np.abs(1-centroid_labels)

    clean_center = embedding[labels==0].mean(axis=0)

    X_plot = np.linspace(0, 1, 1000)[:, np.newaxis]
    kde = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(scores.reshape(len(scores), 1))
    log_dens = kde.score_samples(X_plot)
    i = np.argmax(np.exp(log_dens))
    dense = X_plot[i, 0]

    # ground truth
    fig = plt.figure()
    ax = fig.add_subplot(111,)
    ax.scatter(
        embedding[:,0],
        embedding[:,1],
        s=.1,
        c=clean_label[cls],
        cmap="tab10")
    ax.scatter(
        centroid[:, 0],
        centroid[:, 1],
        s=5,
        c="black")
    plt.show()

    pca_scores = np.abs(scores[idxs]-dense).squeeze()

    clean_centers = np.array([clean_center])
    umap_scores = np.linalg.norm(centroid-clean_center, axis=1)
    umap_scores = umap_scores/umap_scores.max()

    s = umap_scores+pca_scores

    # hybrid scoring
    l=np.zeros(len(centroid))
    l[s>.5] = 1
    plt.scatter(
        centroid[:, 0],
        centroid[:, 1],
        s=5,
        c=s,
        cmap="Reds"
    )
    plt.show()
    plt.scatter(
        centroid[:, 0],
        centroid[:, 1],
        s=5,
        c=l,
        cmap="tab10"
    )
    plt.show()



In [ ]:
s = 0
embedding_2d = embeddings_2d[:,s:]
embeddings_2d.shape,embedding_2d.shape

In [ ]:
for cls_num in range(10):
    cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
    high_data = embedding_2d[cls].reshape(len(cls), -1)

    # umap reduction
    reducer = umap.UMAP(n_components=2)
    embedding = reducer.fit_transform(high_data)
    
    # PCA
    _, _, v = np.linalg.svd(high_data)
    scores = np.abs(np.inner(v[0], high_data))
    m = embedding.mean()
    scores = scores / scores.max()

    brc = Birch(n_clusters=2)
    brc.fit(embedding)
    s = silhouette_score(embedding, brc.labels_, metric='euclidean')
    c = calinski_harabasz_score(embedding, brc.labels_)
    print("silhouette_score\t", s)
    print("calinski_harabasz_score\t", c)
    labels = brc.labels_
    centroid = brc.subcluster_centers_
    centroid_labels = brc.subcluster_labels_

    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(embedding)
    _, idxs = nbrs.kneighbors(centroid)
    centroid = embedding[idxs].squeeze(axis=1)

    # clean 0, noise 1
    if np.sum(labels==0)<np.sum(labels==1):
        labels = np.abs(1-labels)
        centroid_labels = np.abs(1-centroid_labels)
    
    clean_center = embedding[labels==0].mean(axis=0)

    X_plot = np.linspace(0, 1, 1000)[:, np.newaxis]
    kde = KernelDensity(kernel='gaussian', bandwidth=0.75).fit(scores.reshape(len(scores), 1))
    log_dens = kde.score_samples(X_plot)
    i = np.argmax(np.exp(log_dens))
    dense = np.exp(log_dens)[i]

    # ground truth
    fig = plt.figure()
    ax = fig.add_subplot(111,)
    ax.scatter(
        embedding[:,0],
        embedding[:,1],
        s=.1,
        c=clean_label[cls],
        cmap="tab10")
    ax.scatter(
        centroid[:, 0],
        centroid[:, 1],
        s=5,
        c="black")
    plt.show()

    umap_scores = np.linalg.norm(centroid-clean_center, axis=1)
    umap_scores = umap_scores/umap_scores.max()

    log_dens = kde.score_samples(scores.reshape(len(scores), 1))[idxs]
    pca_scores = np.abs(np.exp(log_dens)-dense).squeeze(axis=1)

    clean_scores = pca_scores[np.argwhere(centroid_labels==0)].squeeze(axis=1)
    detect_n = (clean_scores-clean_scores.mean())/clean_scores.std()
    susp_ones = np.argwhere(np.abs(detect_n)>2)
    s = umap_scores+pca_scores
    print(s)

    # umap clustering
    plt.scatter(
        centroid[:, 0],
        centroid[:, 1],
        s=5,
        c=centroid_labels,
        cmap="tab10"
    )
    plt.show()
    # hybrid scoring
    # l = np.copy(centroid_labels)
    # l[np.argwhere(centroid_labels==0)[susp_ones]] = 2
    l=np.zeros(len(centroid))
    l[s>.5] = 1
    plt.scatter(
        centroid[:, 0],
        centroid[:, 1],
        s=5,
        c=s,
        cmap="Reds"
    )
    plt.show()
    plt.scatter(
        centroid[:, 0],
        centroid[:, 1],
        s=5,
        c=l,
        cmap="tab10"
    )
    plt.show()



In [ ]:
s = 85
embedding_2d = embeddings_2d[:,s:]
embeddings_2d.shape,embedding_2d.shape

In [ ]:
for cls_num in range(10):

    cls = np.argwhere(np.array(train_labels)==cls_num).squeeze()
    high_data = embedding_2d[cls].reshape(len(cls), -1)

    _, _, v = np.linalg.svd(high_data)
    scores = np.abs(np.inner(v[0], high_data))
    scores = scores / scores.max()

    reducer = umap.UMAP(n_components=2)
    embedding = reducer.fit_transform(high_data)
    brc = Birch(n_clusters=2)
    brc.fit(embedding)
    s = silhouette_score(embedding, brc.labels_, metric='euclidean')
    c = calinski_harabasz_score(embedding, brc.labels_)
    print("silhouette_score\t", s)
    print("calinski_harabasz_score\t", c)
    # labels = brc.labels_
    # centroid = brc.subcluster_centers_
    # centroid_labels = brc.subcluster_labels_

    fig = plt.figure()
    ax = fig.add_subplot(111,)

    ax.scatter(
        embedding[:,0],
        embedding[:,1],
        # scores,
        s=.1,
        c=train_labels[cls],
        cmap="tab10")

    plt.show()
    plt.scatter(
        embedding[:,0],
        scores,
        s=.1,
        c=train_labels[cls],
        cmap="tab10")

    plt.show()

In [ ]:
_, _, v = np.linalg.svd(high_data)
scores2 = np.abs(np.inner(v[1], high_data))

In [ ]:
from sklearn.decomposition import PCA
reducer = PCA(n_components=1)
s1 = -reducer.fit_transform(high_data)

In [ ]:
reducer = umap.UMAP(n_components=1)
s2 = -reducer.fit_transform(high_data)

In [ ]:
from scipy.stats import pearsonr
pearsonr(scores.squeeze(), s1.squeeze())

In [ ]:
from sklearn.mixture import GaussianMixture as GMM
feats_ = np.ravel(scores1).astype(float).reshape(-1, 1)
gmm = GMM(n_components=2, covariance_type='full', tol=1e-6, max_iter=100)
gmm.fit(feats_)
prob = gmm.predict_proba(feats_)
cl = gmm.predict(feats_)
# prob = prob[:,gmm.means_.argmax()]
means = gmm.means_
import scipy
centers = np.zeros(2, dtype=np.int32)
for i in range(gmm.n_components):
    density = scipy.stats.multivariate_normal(cov=gmm.covariances_[i], mean=gmm.means_[i]).logpdf(feats_)
    centers[i] = int(np.argmax(density))


In [ ]:
centers

In [ ]:
reducer = umap.UMAP(n_components=2)
embedding = reducer.fit_transform(high_data)
plt.title("Ground Truth")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=clean_label[cls],
    cmap="tab10")
plt.show()
plt.title("svd")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=cl,
    cmap="Greens")
plt.scatter(
    embedding[centers][:, 0],
    embedding[centers][:, 1],
    s=5,
    c="black",
    cmap="Greens")
plt.show()
plt.title("svd")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=scores1,
    cmap="Greens")
plt.scatter(
    embedding[centers][:, 0],
    embedding[centers][:, 1],
    s=5,
    c="black",
    cmap="Greens")
plt.show()
plt.title("pca")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=s1,
    cmap="Greens")
plt.scatter(
    embedding[centers][:, 0],
    embedding[centers][:, 1],
    s=5,
    c="black",
    cmap="Greens")
plt.show()
plt.title("umap")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=s2,
    cmap="Greens")
plt.scatter(
    embedding[centers][:, 0],
    embedding[centers][:, 1],
    s=5,
    c="black",
    cmap="Greens")
plt.show()

In [ ]:
reducer = umap.UMAP(n_components=2)
embedding = reducer.fit_transform(high_data)
plt.title("Ground Truth")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=clean_label[cls],
    cmap="tab10")
plt.show()
plt.title("svd")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=scores,
    cmap="Greens")
plt.show()
plt.title("pca")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=s1,
    cmap="Greens")
plt.show()
plt.title("umap")
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=s2,
    cmap="Greens")
plt.show()


In [ ]:
reducer = umap.UMAP(n_components=10)
embedding = reducer.fit_transform(high_data)

feats_ = np.concatenate((scores.reshape(len(scores), 1), embedding), axis=1)
from sklearn import cluster
kmeans = cluster.KMeans(n_clusters=2, random_state=0).fit(feats_)
if np.mean(scores[kmeans.labels_==0]) < np.mean(scores[kmeans.labels_==1]): kmeans.labels_ = 1 - kmeans.labels_  

reducer = PCA(n_components=2)
embedding = reducer.fit_transform(high_data)    



plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=clean_label[cls],
    cmap="tab10")
plt.show()
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=kmeans.labels_,
    cmap="tab10")
plt.show()

In [ ]:
for cls_num in range(10):

    cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
    high_data = embeddings_2d[cls].reshape(len(cls), -1)

    from sklearn.decomposition import PCA
    # G = high_data.T.dot(high_data)

    reducer = PCA(n_components=2)
    embedding = reducer.fit_transform(high_data)
    eigen_vec = reducer.components_
    scores = cosine_similarity(eigen_vec,high_data)
    from sklearn.mixture import GaussianMixture
    gm = GaussianMixture(n_components=2, random_state=0).fit(embedding)
    labels = gm.predict(embedding)
    prob = gm.predict_proba(embedding)

    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=clean_label[cls],
        cmap="tab10")
    plt.show()

    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=prob[:,0],
        cmap="Greens")
    plt.show()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=prob[:,1],
        cmap="Reds")
    plt.show()


    

    #     # reducer = umap.UMAP(n_components=2)
    #     # embedding = reducer.fit_transform(high_data)

    # plt.scatter(
    #     embedding[:, 0],
    #     embedding[:, 1],
    #     s=1,
    #     c=similarity[0],
    #     cmap="Greens")

    # plt.show()
    #     # plt.scatter(
    #     #     embedding[:, 0],
    #     #     embedding[:, 1],
    #     #     s=1,
    #     #     c=clean_label[cls],
    #     #     cmap="tab10")
    #     # plt.show()

    
    # plt.scatter(
    #     embedding[:, 0],
    #     embedding[:, 1],
    #     s=1,
    #     c=labels,
    #     cmap="tab10")

    # plt.show()

In [ ]:
prob.shape

In [ ]:
for cls_num in range(10):
    cls = np.argwhere(np.array(noisy_label)==cls_num).squeeze()
    high_data = embeddings_2d[cls].reshape(len(cls), -1)


    reducer = umap.UMAP(n_components=2)
    embedding = reducer.fit_transform(high_data)

    # from sklearn.manifold import TSNE
    # embedding = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(high_data)

    brc = Birch(n_clusters=2)
    brc.fit(embedding)

    labels = brc.labels_
    centroid = brc.subcluster_centers_
    centroid_labels = brc.subcluster_labels_
    # clean 1, noise 0
    bin = np.bincount(labels)
    if bin[0] > bin[1]:
        centroid_labels = np.abs(centroid_labels-1)
        labels = np.abs(labels-1)

    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=clean_label[cls],
        cmap="tab10")
    plt.scatter(
        brc.subcluster_centers_[:, 0],
        brc.subcluster_centers_[:, 1],
        s=5,
        c='black')
    plt.show()
    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=brc.labels_,
        cmap="Pastel2")

    nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centroid[centroid_labels==1])
    dists, indices = nbrs.kneighbors(centroid[centroid_labels==1])
    suspicious = (dists[:, -1]/ dists[:, 1])>1.8

    cleans = centroid[centroid_labels==1]
    noises = centroid[centroid_labels==0]
    plt.scatter(
        cleans[:, 0],
        cleans[:, 1],
        s=5,
        c='r')
    plt.scatter(
        noises[:, 0],
        noises[:, 1],
        s=5,
        c='black')
    plt.scatter(
        cleans[suspicious][:, 0],
        cleans[suspicious][:, 1],
        s=5,
        c='b')
    plt.show()

In [ ]:
for cls_num in range(10):
    cls = np.argwhere(np.array(train_labels)==cls_num).squeeze()
    high_data = embeddings_2d[cls].reshape(len(cls), -1)


    reducer = umap.UMAP(n_components=2)
    embedding = reducer.fit_transform(high_data)

    # from sklearn.manifold import TSNE
    # embedding = TSNE(n_components=2, learning_rate='auto', init='random').fit_transform(high_data)

    brc = Birch(n_clusters=2)
    brc.fit(embedding)

    labels = brc.labels_
    centroid = brc.subcluster_centers_
    centroid_labels = brc.subcluster_labels_
    # clean 1, noise 0
    bin = np.bincount(labels)
    if bin[0] > bin[1]:
        centroid_labels = np.abs(centroid_labels-1)
        labels = np.abs(labels-1)

    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=train_labels[cls],
        cmap="tab10")
    plt.scatter(
        brc.subcluster_centers_[:, 0],
        brc.subcluster_centers_[:, 1],
        s=5,
        c='black')
    plt.show()
    plt.clf()
    plt.scatter(
        embedding[:, 0],
        embedding[:, 1],
        s=1,
        c=brc.labels_,
        cmap="Pastel2")

    nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centroid[centroid_labels==1])
    dists, indices = nbrs.kneighbors(centroid[centroid_labels==1])
    suspicious = (dists[:, -1]/ dists[:, 1])>1.8

    cleans = centroid[centroid_labels==1]
    noises = centroid[centroid_labels==0]
    plt.scatter(
        cleans[:, 0],
        cleans[:, 1],
        s=5,
        c='r')
    plt.scatter(
        noises[:, 0],
        noises[:, 1],
        s=5,
        c='black')
    plt.scatter(
        cleans[suspicious][:, 0],
        cleans[suspicious][:, 1],
        s=5,
        c='g')
    plt.show()

In [ ]:
plt.clf()
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    s=1,
    c=brc.labels_,
    cmap="Pastel2")

nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(centroid[centroid_labels==1])
dists, indices = nbrs.kneighbors(centroid[centroid_labels==1])
suspicious = (dists[:, -1]/ dists[:, 1])>1.8

cleans = centroid[centroid_labels==1]
noises = centroid[centroid_labels==0]
plt.scatter(
    cleans[:, 0],
    cleans[:, 1],
    s=5,
    c='r')
plt.scatter(
    noises[:, 0],
    noises[:, 1],
    s=5,
    c='black')
plt.scatter(
    cleans[suspicious][:, 0],
    cleans[suspicious][:, 1],
    s=5,
    c='b')
plt.show()